## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [1]:
import random

In [2]:
N = 1000000
A = [random.randint(0, 1000) for _ in range(N)]
B = [a + 100 for a in A]

sr = sum(B) / N

print("Среднее значение массива:", sr)

Среднее значение массива: 599.737329


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [3]:
import pandas as pd

In [6]:
df = pd.DataFrame({
    'col1': [random.randint(0, 100) for _ in range(2000000)],
    'col2': [random.randint(0, 100) for _ in range(2000000)],
    'col3': [random.randint(0, 100) for _ in range(2000000)],
    'col4': [random.randint(0, 100) for _ in range(2000000)]
})

alphabet = 'abcdefghijklmnopqrstuvwxyz'
df['key'] = [''.join(random.choices(alphabet, k=5)) for _ in range(2000000)]
subset = df[df['key'].str[:5].isin(list(alphabet)[:5])]

print(subset)

Empty DataFrame
Columns: [col1, col2, col3, col4, key]
Index: []


## Лабораторная работа 3

In [7]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 10.6 MB/s eta 0:00:00


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [9]:
recipes = pd.read_csv('recipes_sample.csv', index_col=0)
reviews = pd.read_csv('reviews_sample.csv', index_col=0)
reviews['date'] = pd.to_datetime(reviews['date'])
reviews['rating'] = reviews['rating'].astype(float)
def mean_rating_a(reviews):
    total = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year == 2010:
            total += row['rating']
            count += 1
    if count == 0:
        return 0
    return total / count

In [10]:
reviews_2010 = reviews[reviews['date'].dt.year == 2010]

def mean_rating_b(reviews_2010):
    total = 0
    count = 0
    for index, row in reviews_2010.iterrows():
        total += row['rating']
        count += 1
    if count == 0:
        return 0
    return total / count

In [11]:
def mean_rating_c(reviews):
    reviews_2010 = reviews[reviews['date'].dt.year == 2010]
    return reviews_2010['rating'].mean()

In [12]:
print(mean_rating_a(reviews)) 
print(mean_rating_b(reviews_2010))
print(mean_rating_c(reviews))

4.4544402182900615
4.4544402182900615
4.4544402182900615


In [13]:
import timeit

print(timeit.timeit(lambda: mean_rating_a(reviews), number=10))
print(timeit.timeit(lambda: mean_rating_b(reviews_2010), number=10))
print(timeit.timeit(lambda: mean_rating_c(reviews), number=10))

59.745576480999716
5.7242836890000035
0.24351388400009455


3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [28]:
from line_profiler import LineProfiler

def get_word_reviews_count(df):
  word_reviews = {}
  for _, row in df.dropna(subset=['review']).iterrows():
      recipe_id, review = row['recipe_id'], row['review']
      words = review.split(' ')
      for word in words:
          if word not in word_reviews:
              word_reviews[word] = []
          word_reviews[word].append(recipe_id)
  
  word_reviews_count = {}
  for _, row in df.dropna(subset=['review']).iterrows():
      review = row['review']
      words = review.split(' ')
      for word in words:
          word_reviews_count[word] = len(word_reviews[word])
  return word_reviews_count

lp = LineProfiler()
lp_wrapper = lp(get_word_reviews_count)
lp_wrapper(reviews)
lp.print_stats()

Timer unit: 1e-09 s

Total time: 43.4747 s
File: <ipython-input-28-727cd87a6db6>
Function: get_word_reviews_count at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def get_word_reviews_count(df):
     4         1       1192.0   1192.0      0.0    word_reviews = {}
     5    126679 11363186761.0  89700.6     26.1    for _, row in df.dropna(subset=['review']).iterrows():
     6    126679 2579205681.0  20360.2      5.9        recipe_id, review = row['recipe_id'], row['review']
     7    126679  606904554.0   4790.9      1.4        words = review.split(' ')
     8   6792010 1624759165.0    239.2      3.7        for word in words:
     9   6617066 3148232715.0    475.8      7.2            if word not in word_reviews:
    10    174944   90047803.0    514.7      0.2                word_reviews[word] = []
    11   6792010 4090378124.0    602.2      9.4            word_reviews[word].append(recipe_id)
    12                

При анализе результатов выполнения профайлера можно заметить, что основное узкое место - это два вложенных цикла в функции get_word_reviews_count. Каждый раз происходит поиск всех отзывов, содержащих слово, что занимает много времени. Также заметно, что создание списка recipe_id для каждого слова в отзыве не является оптимальным, т.к. достаточно посчитать количество уникальных значений.

Для оптимизации функции можно использовать структуру данных Counter из библиотеки collections, которая позволяет считать количество уникальных элементов в списке. Также можно создать словарь word_reviews_count, в котором хранить количество уникальных recipe_id для каждого слова, чтобы не производить повторный поиск в списке word_reviews.

In [27]:
from collections import Counter

def get_word_reviews_count_optimized(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)

    word_reviews_count = {}
    for word, recipe_ids in word_reviews.items():
        word_reviews_count[word] = len(set(recipe_ids))

    return word_reviews_count

lp = LineProfiler()
lp_wrapper = lp(get_word_reviews_count_optimized)
lp_wrapper(reviews)
lp.print_stats()

Timer unit: 1e-09 s

Total time: 23.6691 s
File: <ipython-input-27-73b3360df04d>
Function: get_word_reviews_count_optimized at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def get_word_reviews_count_optimized(df):
     4         1       1346.0   1346.0      0.0      word_reviews = {}
     5    126679 11042192969.0  87166.7     46.7      for _, row in df.dropna(subset=['review']).iterrows():
     6    126679 2567199550.0  20265.4     10.8          recipe_id, review = row['recipe_id'], row['review']
     7    126679  598761534.0   4726.6      2.5          words = review.split(' ')
     8   6792010 1558786771.0    229.5      6.6          for word in words:
     9   6617066 3168036458.0    478.8     13.4              if word not in word_reviews:
    10    174944   96255538.0    550.2      0.4                  word_reviews[word] = []
    11   6792010 4051107180.0    596.5     17.1              word_reviews[word].appe

После оптимизации кода мы можем увидеть, что время выполнения сократилось до 23 секунд. (Вместо 43)

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [38]:
#Без использования векторизованных операций и методов массивов numpy и без использования numba:
def mape1(ratings):
    n = len(ratings)
    mean_rating = sum(ratings) / n
    mape = 0
    count = 0
    for rating in ratings:
        if rating != 0:
            mape += abs(rating - mean_rating) / rating
            count += 1
    if count > 0:
        mape /= count
    return mape * 100

In [39]:
from line_profiler import LineProfiler

lp = LineProfiler()
lp.add_function(mape1)
lp_wrapper = lp(mape1)

ratings = [1, 2, 3, 4, 5]
lp_wrapper(ratings)
lp.print_stats()

Timer unit: 1e-09 s

Total time: 1.6524e-05 s
File: <ipython-input-38-42d1699bf53a>
Function: mape1 at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def mape1(ratings):
     3         1       1606.0   1606.0      9.7      n = len(ratings)
     4         1       1924.0   1924.0     11.6      mean_rating = sum(ratings) / n
     5         1        308.0    308.0      1.9      mape = 0
     6         1        182.0    182.0      1.1      count = 0
     7         5       1743.0    348.6     10.5      for rating in ratings:
     8         5       1845.0    369.0     11.2          if rating != 0:
     9         5       5214.0   1042.8     31.6              mape += abs(rating - mean_rating) / rating
    10         5       2202.0    440.4     13.3              count += 1
    11         1        360.0    360.0      2.2      if count > 0:
    12         1        498.0    498.0      3.0          mape /= count
    13         

In [40]:
#Без использования векторизованных операций и методов массивов numpy, но с использованием numba:
from numba import jit

@jit(nopython=True)
def mape2(ratings):
    n = len(ratings)
    mean_rating = sum(ratings) / n
    mape = 0
    count = 0
    for rating in ratings:
        if rating != 0:
            mape += abs(rating - mean_rating) / rating
            count += 1
    if count > 0:
        mape /= count
    return mape * 100

In [42]:
from line_profiler import LineProfiler

lp = LineProfiler()
lp.add_function(mape2)
lp_wrapper = lp(mape2)

ratings = [1, 2, 3, 4, 5]
lp_wrapper(ratings)
lp.print_stats()

Timer unit: 1e-09 s

Total time: 0 s
File: <ipython-input-40-35bbfd014744>
Function: mape2 at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           @jit(nopython=True)
     5                                           def mape2(ratings):
     6                                               n = len(ratings)
     7                                               mean_rating = sum(ratings) / n
     8                                               mape = 0
     9                                               count = 0
    10                                               for rating in ratings:
    11                                                   if rating != 0:
    12                                                       mape += abs(rating - mean_rating) / rating
    13                                                       count += 1
    14                                               if count > 0:
    15                    

<ipython-input-42-1ba3dbc345cc>:4: UserWarning: Adding a function with a __wrapped__ attribute. You may want to profile the wrapped function by adding mape2.__wrapped__ instead.
  lp.add_function(mape2)
/usr/local/lib/python3.10/dist-packages/line_profiler/line_profiler.py:55: UserWarning: Adding a function with a __wrapped__ attribute. You may want to profile the wrapped function by adding mape2.__wrapped__ instead.
  self.add_function(func)


In [44]:
#С использованием векторизованных операций и методов массивов numpy, но без использования numba:
import numpy as np

def mape3(ratings):
    ratings = np.array(ratings)
    mean_rating = np.mean(ratings)
    mask = (ratings != 0)
    mape = np.mean(np.abs(ratings - mean_rating)[mask] / ratings[mask]) * 100
    return mape

In [45]:
from line_profiler import LineProfiler

lp = LineProfiler()
lp.add_function(mape3)
lp_wrapper = lp(mape3)

ratings = [1, 2, 3, 4, 5]
lp_wrapper(ratings)
lp.print_stats()

Timer unit: 1e-09 s

Total time: 0.000222175 s
File: <ipython-input-44-fab7c50dc54b>
Function: mape3 at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def mape3(ratings):
     5         1      20958.0  20958.0      9.4      ratings = np.array(ratings)
     6         1     107555.0 107555.0     48.4      mean_rating = np.mean(ratings)
     7         1      17102.0  17102.0      7.7      mask = (ratings != 0)
     8         1      76257.0  76257.0     34.3      mape = np.mean(np.abs(ratings - mean_rating)[mask] / ratings[mask]) * 100
     9         1        303.0    303.0      0.1      return mape



In [47]:
#C использованием векторизованных операций и методов массивов numpy и numba:
from numba import jit

@jit(nopython=True)
def mape4(ratings):
    ratings = np.array(ratings)
    mean_rating = np.mean(ratings)
    mask = (ratings != 0)
    mape = np.mean(np.abs(ratings - mean_rating)[mask] / ratings[mask]) * 100
    return mape

In [51]:
from line_profiler import LineProfiler

lp = LineProfiler()
lp.add_function(mape4)
lp_wrapper = lp(mape4)

ratings = [1, 2, 3, 4, 5]
lp_wrapper(ratings)
lp.print_stats()

Timer unit: 1e-09 s

Total time: 0 s
File: <ipython-input-47-62f1ba6e5e67>
Function: mape4 at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           @jit(nopython=True)
     5                                           def mape4(ratings):
     6                                               ratings = np.array(ratings)
     7                                               mean_rating = np.mean(ratings)
     8                                               mask = (ratings != 0)
     9                                               mape = np.mean(np.abs(ratings - mean_rating)[mask] / ratings[mask]) * 100
    10                                               return mape



<ipython-input-51-8d710028b6ea>:4: UserWarning: Adding a function with a __wrapped__ attribute. You may want to profile the wrapped function by adding mape4.__wrapped__ instead.
  lp.add_function(mape4)


#### [версия 2]
* Уточнены формулировки задач 1, 3, 4